# WEEK-2 Assignment(By-Ayush Paul)

##### The following code is totally experimental and may be changed down the line for better accuracy.

## Bank Dataset

In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

In [56]:
bank_data_full=pd.read_csv('C:/Users/Admin/Desktop/ACM/ML LOC MENTOR/bank-full.csv')
bank_data_full.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [57]:
#Fetching the Prediction Target
y=bank_data_full.y
y

0         no
1         no
2         no
3         no
4         no
        ... 
45206    yes
45207    yes
45208    yes
45209     no
45210     no
Name: y, Length: 45211, dtype: object

## Function-1

In [58]:
def get_data_splits(X_full,train_perc):
    #Checking to see if the given dataset has or has not the prediction target column
    try: 
        #Dropping the Prediction Target Column
        X_full=bank_data_full.drop(columns=['y']).copy()
    except:
        X_full=bank_data_full
        #Splitting the data into test and validation dataset
        X_train_full,X_test_full,y_train,y_valid=train_test_split(X_full,y,train_size=train_perc)
        #Selecting categorical columns for OneHotEncoding
        categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique()<13 and X_train_full[cname].dtype == "object"]

        #Selecting numerical columns for SimpleImputation
        numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64','float64']]
        # Keep selected columns only
        my_cols = categorical_cols + numerical_cols

        #Merge the selected columns for the Final Training and Validation DataSet
        X_train = X_train_full[my_cols].copy()
        X_valid = X_test_full[my_cols].copy()
        return X_train,X_valid,y_train,y_valid
    else:
        #Splitting the data into test and validation dataset
        X_train_full,X_test_full,y_train,y_valid=train_test_split(X_full,y,train_size=train_perc)
        #Selecting categorical columns for OneHotEncoding
        categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique()<13 and X_train_full[cname].dtype == "object"]

        #Selecting numerical columns for SimpleImputation
        numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64','float64']]
        # Keep selected columns only
        my_cols = categorical_cols + numerical_cols

        #Merge the selected columns for the Final Training and Validation DataSet
        X_train = X_train_full[my_cols].copy()
        X_valid = X_test_full[my_cols].copy()
        return X_train,X_valid,y_train,y_valid

## Function-2

In [59]:
def train_model(X_train,y_train):
    #Preprocessing for numerical data.Using Simple Imputer to replace 
    #NaN values with the mean value of the particular column
    numerical_transformer = SimpleImputer(strategy='mean')
    # Preprocessing for categorical data.When the OneHotEncoder parameter is set to ‘ignore’ and an 
    #unknown category is encountered during transform, the resulting one-hot encoded 
    #columns for this feature will be all zeros
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    # Bundle preprocessing for numerical and categorical data
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])
    #Initializing Value of K for KNN Algorithm
    k=int(round(np.sqrt(X_train.shape[0])))
    if k%2==0:
        k=k+1
    #Define Model
    model = KNeighborsClassifier(n_neighbors=k)
    # Bundle preprocessing and modeling code in a pipeline
    my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                  ('model', model)
                                 ])
    # Preprocessing of training data, fit model 
    my_pipeline.fit(X_train, y_train)
    return my_pipeline
    
    

## Function-3

In [60]:
def get_model_score(model,X_valid,y_valid):
    """Function Defintion --- The function takes three parameters model(the trained model),the validation
       or test data and the prediction target for calculation of accuracy score. It returns the prediction
       as an numpy array."""
    #Cross-Validation with 7 folds
    cv_scores = cross_val_score(model, X_valid,y_valid, 
                            cv=7,
                            scoring='accuracy')
    preds=model.predict(X_valid)
    print("Confusion Matrix :") 
    print(confusion_matrix(y_valid,preds)) #Print Confusion Matrix
    print()
    print("Cross-validation accuracy: %f" % cv_scores.mean()) #Print Cross-Validation Score
    return preds #return predictions

## Driver Code

In [61]:
train_X,valid_X,train_y,valid_y=get_data_splits(X_full,0.8)
train_y

18076     no
13807    yes
15701     no
9493      no
16971     no
        ... 
42088    yes
8762      no
15402     no
34780     no
21800     no
Name: y, Length: 36168, dtype: object

In [62]:
my_model=train_model(train_X,train_y)
my_model

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='mean',
                                                                verbose=0),
                                                  ['age', 'balance', 'day',
                                                   'duration', 'campaign',
                                                   'pdays', 'previous']

In [63]:
prediction=get_model_score(my_model,valid_X,valid_y)

Confusion Matrix :
[[7918   82]
 [ 923  120]]

Cross-validation accuracy: 0.885989


In [64]:
prediction

array(['no', 'no', 'no', ..., 'no', 'no', 'no'], dtype=object)